# Grab historical weather data & elevation form addresses in California

In [1]:
#Import custom Google Maps and Weatherbit utlity functions
from utility import *
import pandas as pd
from tabulate import tabulate

## Import csv of California vineyard locations
http://www.discovercaliforniawines.com provides a directory of vineyards in California. I used this directory to collect addresses of vineyards around California. 

Some of the addresses listed were showrooms, so I filtered those out of the list since we only want to collect information on where the grapes are grown.

## Adding in non-vineyards
I then added in addresses of locations in California which aren't vineyards. I realize that this isn't perfect because a **certain location might be great for a vineyard, but just doesn't happen to have one located there.** Because of this, I don't expect my model to reach extremley high accuracy results because the data will be a bit noisy. 

In [2]:
#Import the csv of addresses (shuffle the rows)
address_df = pd.read_csv('california_vineyards.csv', encoding='cp1252').sample(frac=1)

#Pretty print the address dataframe
print(tabulate(address_df.head(10), headers=['Address', 'Name', 'Address'], tablefmt= 'grid'))

+-----+-----------+---------------------------------+--------------------------------------------+
|     |   Address | Name                            | Address                                    |
+=====+===========+=================================+============================================+
|  45 |         0 | nan                             | 4971 Rocky Rd, San Andreas, CA 95249       |
+-----+-----------+---------------------------------+--------------------------------------------+
| 635 |         1 | Heritage Oak Winery             | 10112 E. Woodbridge Road, Acampo, CA 95220 |
+-----+-----------+---------------------------------+--------------------------------------------+
| 405 |         0 | nan                             | 0 Paseo Chaparro, Murrieta, CA 92562       |
+-----+-----------+---------------------------------+--------------------------------------------+
| 884 |         1 | William Hill Estate Winery      | 1761 Atlas Peak Road, Napa, CA 94558       |
+-----+---

# Use Google Maps & WeatherBit APIs to gather data on addresses
Note that both these APIs have rate limits. To run this yourself, you'll need to reduce the number of addresses passed to the land_data() function or upgrade from basic accounts with Google Maps & Weatherbit.

## Google Maps data collected
* Latitude & longitude
* Matrix of elevation points based on 1km area around latitude & longitude coordinates

## Weatherbit data collected
* Wind direction
* Wind speed
* Precipitation
* Average temperature
* Minimum temperature
* Max temperature
* Cloud coverage
* GHI (Global Horizontal Irradiance) - aka solar radiation
* RH (Relative humidity)

In [10]:
def land_data(df, new_df):
    import numpy as np
    import time
    from datetime import timedelta
    
    year_offset = timedelta(days=364) #I know, I know, there aren't 364 days in a year, but this accounts for leapyears + my limited 1year of historical data
    end_date = pd.to_datetime('today')
    start_date = end_date - year_offset

    #1-year of dates as list
    d = pd.date_range(start=start_date, end=end_date, freq='D')

    #A variable to store the last date to use in the range of the weather api data
    last_date_str = 0

    for index, row in df.iterrows():
        address = row['Address']
                
        #First check to see if this address already exists
        if(elevation_weather_df['Address'].str.contains(address).any() == False):

            #if(elevation_weather_df['Address'].str.contains(address).any() == False):
            print('Collecting data for address: ' + str(address))

            #Get numerical latitute and longitude values
            lat, lng = lat_lng(address)

            #Create blank arrays to store weather data for each address
            wind_dir = np.array([])
            wind_spd = np.array([])
            precip = np.array([])
            temp = np.array([])
            min_temp = np.array([])
            max_temp = np.array([])
            clouds = np.array([])
            ghi = np.array([])
            rh = np.array([])

            elevation = np.array([elevation_matrix(lat, lng)])

            #Collect weather data for all dates over the last year
            for date in d[:]:

                #format the date as a string - truncate to the first 10 characters
                date_str = str(date)[:10]  

                #Get day as int
                day = int(date_str[-2:])

                #Grab data every 5 days
                skip_days = 5

                if ((last_date_str != 0) & (day % skip_days == 0)):

                    #Get a dictionary of weather data based off a day
                    weather_data = weather_hist(last_date_str, date_str, lat, lng)

                    #Grab elements from the weather_data dictionary
                    #address_elevation = np.array([elevation_matrix(lat, lng)])
                    wind_dir = np.append(wind_dir, weather_data.get('wind_dir'))
                    wind_spd = np.append(wind_spd, weather_data.get('wind_spd'))
                    precip = np.append(precip, weather_data.get('precip'))
                    temp = np.append(temp, weather_data.get('temp'))
                    min_temp = np.append(min_temp, weather_data.get('min_temp'))
                    max_temp = np.append(max_temp, weather_data.get('max_temp'))
                    clouds = np.append(clouds, weather_data.get('clouds'))
                    ghi = np.append(ghi, weather_data.get('ghi'))
                    rh = np.append(rh, weather_data.get('rh'))

                #Save this date to be used as the start date for the next API call
                last_date_str = date_str

            #Update the new land details dataframe
            new_df.loc[len(new_df)] = row['Vineyard'], row['Address'], row['Name'], (lat, lng), elevation, wind_dir, wind_spd, precip, temp, min_temp, max_temp, clouds, ghi, rh 

        last_date_str = 0
        
        #The pkl file is used by build_vineyard_scout.ipynb to build our predictive model
        with open('california_vineyards_elevation_weather.pkl', 'wb') as f: 
            pickle.dump(elevation_weather_df, f)

        #The csv is exported as a friendly format in case we want to view the data outside of python
        new_df.to_csv('california_vineyards_elevation_weather.csv', index=False)
        

## Store elevation & weather data in a dataframe
Building this information takes a couple hours with multiple API calls. Because of this, I set the file up to allow for interuptions in that process. First, see if there's already a saved pkl file of the dataframe, skip any duplicated rows, and then build off of that same dataframe.

Update the pkl and csv file on each loop of an address

In [11]:
import pickle

#First see if there's a saved version of the dataframe
try:
    with open('california_vineyards_elevation_weather.pkl', 'rb') as f: 
        elevation_weather_df = pickle.load(f)
        
    print('Found saved pkl file of weather & elevation data to build off of...')
        
#If that file doesn't exist, create a new dataframe
except Exception:
    print('No existing saved pkl file of weather & elevation data to build off of. Starting from scratch')

    #Columns for blank dataframe
    land_data_cols = ['IsVineyard','Address','Name','MapCoords','ElevationMatrix','WindDir','WindSpd','Precip','Temp', 'MinTemp','MaxTemp','Clouds','GHI','RH']

    #Create blank train and test land dataframes
    elevation_weather_df = pd.DataFrame(columns=land_data_cols)


land_data(address_df, elevation_weather_df)

Found saved pkl file of weather & elevation data to build off of.


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
